# 검색 인덱싱

In [ ]:
import meilisearch as ms
import json

In [ ]:
client = ms.Client('http://localhost:7700/','aSampleMasterKey')

In [50]:
import pandas as pd
data_path = './data/nasdaq_screener_1749629170012.csv'
df = pd.read_csv(data_path, na_filter=False)
df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$120.555,3.8150,3.268%,34245423284.00,United States,1999,1589621,Industrials,Biotechnology: Laboratory Analytical Instruments
1,AA,Alcoa Corporation Common Stock,$28.74,0.1800,0.63%,7440853194.00,United States,2016,2917805,Industrials,Aluminum
2,AACB,Artius II Acquisition Inc. Class A Ordinary Sh...,$10.11,0.0000,0.00%,0.00,United States,2025,37,,
3,AACBR,Artius II Acquisition Inc. Rights,$0.2346,0.0146,6.636%,0.00,United States,2025,1451,,
4,AACBU,Artius II Acquisition Inc. Units,$10.45,0.1177,1.139%,0.00,United States,2025,4825,Finance,Blank Checks


# meilisearch 사용방법
- vector 서치 지원, 클라우드 버전 지원 (유료)

In [51]:
# 전처리 하기
# 특수문자가 포함된 값만 필터링 (정규표현식 사용)
df[df['Symbol'].str.contains(r'[^a-zA-z0-9]', regex=True)]

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
242,AKO/A,Embotelladora Andina S.A.,$19.50,-0.5000,-2.50%,,Chile,,2279,,
243,AKO/B,Embotelladora Andina S.A.,$24.1824,-0.0433,-0.179%,,Chile,,7454,,
781,BF/A,Brown Forman Corporation,$28.59,0.7400,2.657%,,United States,,113773,,
782,BF/B,Brown Forman Corporation,$28.29,0.6300,2.278%,,United States,,2775762,,
842,BIO/B,Bio-Rad Laboratories Inc.,$250.08,0.0000,0.00%,,United States,,16,,
986,BRK/A,Berkshire Hathaway Inc.,$736716.160000000033,-3208.8400,-0.434%,1083616645124.00,United States,,254,,
987,BRK/B,Berkshire Hathaway Inc.,$490.945,-2.3850,-0.483%,1083177536036.00,United States,,1943262,,
1552,CRD/A,Crawford & Company,$10.895,0.0250,0.23%,,United States,,8737,,
1553,CRD/B,Crawford & Company,$10.17,0.3800,3.882%,,United States,,2294,,
1966,ECC,Eagle Point Credit Company Inc. Common Stock,$7.78,-0.1400,-1.768%,471954437.00,United States,2014,1362758,Finance,Trusts Except Educational Religious and Charit...


In [52]:
# r'[/^]'는 정규 표현식(Regex)으로, 문자열 내에서 슬래시(/)나 캐럿(^) 문자를 찾아서 이를 밑줄(_)로 대체함.
df['id'] =df['Symbol'].str.strip().replace(r'[/^]', '_', regex=True)
df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry,id
0,A,Agilent Technologies Inc. Common Stock,$120.555,3.8150,3.268%,34245423284.00,United States,1999,1589621,Industrials,Biotechnology: Laboratory Analytical Instruments,A
1,AA,Alcoa Corporation Common Stock,$28.74,0.1800,0.63%,7440853194.00,United States,2016,2917805,Industrials,Aluminum,AA
2,AACB,Artius II Acquisition Inc. Class A Ordinary Sh...,$10.11,0.0000,0.00%,0.00,United States,2025,37,,,AACB
3,AACBR,Artius II Acquisition Inc. Rights,$0.2346,0.0146,6.636%,0.00,United States,2025,1451,,,AACBR
4,AACBU,Artius II Acquisition Inc. Units,$10.45,0.1177,1.139%,0.00,United States,2025,4825,Finance,Blank Checks,AACBU


# 데이터 인덱싱 하기

In [53]:
# index 생성 (nasdaq 라는 이름으로),  반드시 dict로 변경해야함. 서치 엔진에 업로딩하기
client.index('nasdaq').add_documents(
    df.to_dict(orient="records"), 
    primary_key='id'
)

TaskInfo(task_uid=6, index_uid='nasdaq', status='enqueued', type='documentAdditionOrUpdate', enqueued_at=datetime.datetime(2025, 6, 11, 8, 47, 12, 698087))

In [54]:
# 서치하기
# client.index('nasdaq').search("Apple")
client.index('nasdaq').search("AAPL")

{'hits': [],
 'query': 'AAPL',
 'processingTimeMs': 0,
 'limit': 20,
 'offset': 0,
 'estimatedTotalHits': 0}

In [55]:
# # 인덱스 삭제
# client.delete_index('nasdaq')